<a href="https://colab.research.google.com/github/CharlieKerfoot/finance-agent/blob/main/fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install weave wandb
!pip install --no-deps xformer trl peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-vck9i9w6/unsloth_99b6945888cd45a1aff2ea1e178be633
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-vck9i9w6/unsloth_99b6945888cd45a1aff2ea1e178be633
  Resolved https://github.com/unslothai/unsloth.git to commit 6789c279d578278aca4af22f4ca31fc42829c9a4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import wandb
wandb.login()

wandb: Currently logged in as: ckerf (ckerfteam) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
import weave

run = wandb.init(
    project="Arb-Agent-v1",
    name="llama3-8b-lora-run3",
    job_type="training",
)

wandb: Initializing weave.
weave: wandb version 0.23.1 is available!  To upgrade, please run:
weave:  $ pip install wandb --upgrade
weave: Logged in as Weights & Biases user: ckerf.
weave: View Weave data at https://wandb.ai/ckerfteam/Arb-Agent-v1/weave


In [ ]:
from unsloth import FastLanguageModel

max_seq_length = 8192
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing="unsloth",
    random_state=3407
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

Unsloth 2025.11.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
from datasets import load_dataset

dataset = load_dataset("ckerf/arb-instruct-v1", split="train")

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

def format_prompts(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
      text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
      texts.append(text)
    return {
      "text": texts,
    }

dataset = dataset.map(format_prompts, batched=True)


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/7.47M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4632 [00:00<?, ? examples/s]

Map:   0%|          | 0/4632 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 8192,
    dataset_num_proc = 8,
    packing = True,
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 2,
        num_train_epochs=1,
        warmup_steps = 10,
        learning_rate = 2e-4,
        fp16 = False,
        bf16 = True,
        logging_steps = 25,
        logging_strategy = "steps",
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        report_to = "wandb",
        run_name = "llama3-8b-lora-run2",
        seed = 3407,
        output_dir = "outputs",
    ),
)

trainer_stats = trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/4632 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 4,632 | Num Epochs = 1 | Total steps = 290
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 2 x 1) = 16
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
25,1.490700
50,1.345500
75,1.404600
100,1.341900
125,1.299000
150,1.324700
175,1.283100
200,1.244700
225,1.249300
250,1.241600


train/epoch,▁▂▂▃▄▄▅▆▆▇██
train/global_step,▁▂▂▃▄▄▅▆▆▇██
train/grad_norm,▃▂▁▄▃▆▆█▆▆▇
train/learning_rate,█▇▇▆▅▄▄▃▂▂▁
train/loss,█▄▆▄▃▃▂▁▁▁▁
total_flos,2.1240616912964813e+17
train/epoch,1
train/global_step,290
train/grad_norm,0.28449
train/learning_rate,1e-05
train/loss,1.2357


In [ ]:
FastLanguageModel.for_inference(model)

import random
samples = random.sample(list(dataset), 5)

for sample in samples:
    inputs = tokenizer(
        [
            alpaca_prompt.format(
                sample["instruction"],
                sample["input"],
                "",
            )
        ], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True)

    print("\n" + "-" * 50 + "\n")
    print(f"Q: {sample['instruction']}")
    print(f"A:\n{tokenizer.batch_decode(outputs)[0].split('### Response:')[1]}")
    print(f"Ground Truth:\n{sample['output']}")


--------------------------------------------------

Q: What was the commercial remaining performance obligation compared to the Microsoft Cloud revenue in the first quarter of fiscal year 2024?
A:

The provided text does not contain information about the specific numerical values for the commercial remaining performance obligation and the Microsoft Cloud revenue in the first quarter of fiscal year 2024. It discusses the metrics related to the commercial business, including the definition of the commercial remaining performance obligation, which is the portion of revenue allocated to remaining performance obligations, and the Microsoft Cloud revenue, which includes revenue from Azure and other cloud services, Office 365 Commercial, the commercial portion of LinkedIn, Dynamics 365, and other commercial cloud properties.<|end_of_text|>
Ground Truth:
The provided text does not contain information about the specific figures for commercial remaining performance obligation or Microsoft Cloud

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model.push_to_hub("ckerf/arbagent-llama3-8b-lora")
tokenizer.push_to_hub("ckerf/arbagent-llama3-8b-lora")

README.md:   0%|          | 0.00/572 [00:00<?, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:   0%|          |  557kB /  168MB            

Saved model to https://huggingface.co/ckerf/arbagent-llama3-8b-lora


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...mp6uf75u6n/tokenizer.json: 100%|##########| 17.2MB / 17.2MB            